<a href="https://colab.research.google.com/github/rei-kun01/Simbolo_Webinar_Feb2025/blob/main/%5BSimbolo_webinar%5D_Transformer_simple_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Transformer from scratch with Training

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

# Initialize tokenizers for source (English) and target (French) languages
source_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
target_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # For simplicity, using the same tokenizer


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Example sentence pairs for translation
sentence_pairs = [
    ("Hello, how are you?", "Bonjour, comment ça va?"),
    ("I am fine.", "Je vais bien."),
    ("What is your name?", "Comment tu t'appelles?"),
    ("Good morning.", "Bonjour."),
    ("Thank you.", "Merci.")
]


In [ ]:
# Tokenize the sentences
source_sentences = [pair[0] for pair in sentence_pairs]
target_sentences = [pair[1] for pair in sentence_pairs]

source_tokens = [source_tokenizer.tokenize(sentence) for sentence in source_sentences]
target_tokens = [target_tokenizer.tokenize(sentence) for sentence in target_sentences]

source_ids = [source_tokenizer.convert_tokens_to_ids(tokens) for tokens in source_tokens]
target_ids = [target_tokenizer.convert_tokens_to_ids(tokens) for tokens in target_tokens]

In [ ]:
# Print tokenized sentences
for i in range(len(sentence_pairs)):
    print(f"Source Sentence: {source_sentences[i]}")
    print(f"Source Tokens: {source_tokens[i]}")
    print(f"Source Token IDs: {source_ids[i]}")
    print(f"Target Sentence: {target_sentences[i]}")
    print(f"Target Tokens: {target_tokens[i]}")
    print(f"Target Token IDs: {target_ids[i]}")
    print("-" * 50)


Source Sentence: Hello, how are you?
Source Tokens: ['hello', ',', 'how', 'are', 'you', '?']
Source Token IDs: [7592, 1010, 2129, 2024, 2017, 1029]
Target Sentence: Bonjour, comment ça va?
Target Tokens: ['bon', '##jou', '##r', ',', 'comment', 'ca', 'va', '?']
Target Token IDs: [14753, 23099, 2099, 1010, 7615, 6187, 12436, 1029]
--------------------------------------------------
Source Sentence: I am fine.
Source Tokens: ['i', 'am', 'fine', '.']
Source Token IDs: [1045, 2572, 2986, 1012]
Target Sentence: Je vais bien.
Target Tokens: ['je', 'va', '##is', 'bien', '.']
Target Token IDs: [15333, 12436, 2483, 29316, 1012]
--------------------------------------------------
Source Sentence: What is your name?
Source Tokens: ['what', 'is', 'your', 'name', '?']
Source Token IDs: [2054, 2003, 2115, 2171, 1029]
Target Sentence: Comment tu t'appelles?
Target Tokens: ['comment', 'tu', 't', "'", 'app', '##elles', '?']
Target Token IDs: [7615, 10722, 1056, 1005, 10439, 22869, 1029]
------------------

In [ ]:
import torch
import torch.nn as nn

# Define embedding size
embedding_size = 16  # Increased size for better representation

# Initialize embedding layers
source_embedding = nn.Embedding(num_embeddings=source_tokenizer.vocab_size, embedding_dim=embedding_size)
target_embedding = nn.Embedding(num_embeddings=target_tokenizer.vocab_size, embedding_dim=embedding_size)

# Example: Embed the first source and target sentence
src_tensor = torch.tensor(source_ids[0]).unsqueeze(1)  # Shape: (seq_len, batch_size)
tgt_tensor = torch.tensor(target_ids[0]).unsqueeze(1)  # Shape: (seq_len, batch_size)

src_embeddings = source_embedding(src_tensor)  # Shape: (seq_len, batch_size, embedding_size)
tgt_embeddings = target_embedding(tgt_tensor)

print("Source Embeddings:\n", src_embeddings)
print("\nTarget Embeddings:\n", tgt_embeddings)


Source Embeddings:
 tensor([[[-0.2131, -0.2502,  0.2571,  0.3181, -2.0183, -0.5193, -1.6801,
          -1.7983, -1.1444, -1.0930,  0.7977,  1.2650, -1.5490, -0.9873,
           1.7140,  1.4206]],

        [[ 0.4572,  0.9033,  0.5177,  1.0481, -1.7309, -1.9259, -0.7172,
           1.1693, -0.3357,  1.0813,  0.1753,  0.6464, -0.7173, -1.2524,
          -0.0767, -1.6524]],

        [[ 1.4907,  1.2575, -0.3415, -0.8283, -0.8725, -0.7705, -0.4548,
          -0.6068, -0.7149, -0.5248, -0.6289,  0.6477,  0.2031, -0.2627,
          -0.5148,  0.4083]],

        [[-1.4171,  1.2809, -0.0826,  0.9342,  1.1126,  0.7300,  0.6497,
           1.7969, -1.1044,  0.7705,  0.5696, -0.9654,  0.8908,  0.2298,
           1.2678, -0.6189]],

        [[ 1.0597,  0.5616, -0.5055, -1.5704, -0.4688,  0.7676,  0.7323,
           0.5348, -0.5909, -0.2657, -0.9691, -1.7645, -0.8297,  1.8961,
           0.4147,  0.5618]],

        [[-1.3102,  0.0084, -1.6856,  0.1850,  2.2867, -1.4619, -0.8953,
           0.0626,  1.

In [ ]:
import math

def get_positional_encoding(seq_len, d_model):
    pe = torch.zeros(seq_len, d_model)
    for pos in range(seq_len):
        for i in range(0, d_model, 2):
            pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
            if i + 1 < d_model:
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
    return pe

# Example positional encoding for source and target
src_pos_encoding = get_positional_encoding(src_tensor.size(0), embedding_size)
tgt_pos_encoding = get_positional_encoding(tgt_tensor.size(0), embedding_size)

print("Source Positional Encoding:\n", src_pos_encoding)
print("\nTarget Positional Encoding:\n", tgt_pos_encoding)


Source Positional Encoding:
 tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,
          0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00],
        [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
          9.9999e-01,  1.0000e-03,  1.0000e+00,  1.0000e-04,  1.0000e+00,
          1.0000e-05,  1.0000e+00,  1.0000e-06,  1.0000e+00,  1.0000e-07,
          1.0000e+00],
        [ 9.0930e-01,  8.0658e-01,  1.9867e-01,  9.9800e-01,  1.9999e-02,
          9.9998e-01,  2.0000e-03,  1.0000e+00,  2.0000e-04,  1.0000e+00,
          2.0000e-05,  1.0000e+00,  2.0000e-06,  1.0000e+00,  2.0000e-07,
          1.0000e+00],
        [ 1.4112e-01,  5.8275e-01,  2.9552e-01,  9.9550e-01,  2.9996e-02,
          9.9995e-01,  3.0000e-03,  1.0000e+00,  3.0000e-04,  1.0000e+00,
          3.0000e-05,  1.0000e+00,  3.0000e-06,  1.0000e+00,  3.0000e-07,
          1.00

In [ ]:
import torch.optim as optim

# Define the Transformer model
class SimpleTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embedding_size, num_heads, num_encoder_layers, num_decoder_layers, dim_feedforward=512, dropout=0.1):
        super(SimpleTransformer, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, embedding_size)
        self.pos_encoder = nn.Embedding(1000, embedding_size)  # Assuming max seq_len=1000
        self.pos_decoder = nn.Embedding(1000, embedding_size)
        self.transformer = nn.Transformer(d_model=embedding_size, nhead=num_heads,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout)
        self.fc_out = nn.Linear(embedding_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt):
        src_seq_length = src.size(0)
        tgt_seq_length = tgt.size(0)

        # Create position tensors
        src_positions = torch.arange(0, src_seq_length).unsqueeze(1).to(src.device)
        tgt_positions = torch.arange(0, tgt_seq_length).unsqueeze(1).to(tgt.device)

        # Embed and add positional encoding
        src_emb = self.src_embedding(src) + self.pos_encoder(src_positions)
        tgt_emb = self.tgt_embedding(tgt) + self.pos_decoder(tgt_positions)

        src_emb = self.dropout(src_emb)
        tgt_emb = self.dropout(tgt_emb)

        # Generate masks
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_seq_length).to(src.device)

        # Pass through Transformer
        output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)

        # Output layer
        output = self.fc_out(output)
        return output




In [ ]:
# Initialize the Transformer
transformer_model = SimpleTransformer(
    src_vocab_size=source_tokenizer.vocab_size,
    tgt_vocab_size=target_tokenizer.vocab_size,
    embedding_size=embedding_size,
    num_heads=4,
    num_encoder_layers=2,
    num_decoder_layers=2
)

print(transformer_model)

SimpleTransformer(
  (src_embedding): Embedding(30522, 16)
  (tgt_embedding): Embedding(30522, 16)
  (pos_encoder): Embedding(1000, 16)
  (pos_decoder): Embedding(1000, 16)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    )

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# Define hyperparameters
num_epochs = 1500
learning_rate = 0.001

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer_model.parameters(), lr=learning_rate)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transformer_model = transformer_model.to(device)

# Prepare data
# For simplicity, we'll use the entire dataset as a batch
def prepare_batch(sentence_pairs):
    src_batch = []
    tgt_batch = []
    for src, tgt in sentence_pairs:
        src_ids = source_tokenizer.convert_tokens_to_ids(source_tokenizer.tokenize(src))
        tgt_ids = target_tokenizer.convert_tokens_to_ids(target_tokenizer.tokenize(tgt))
        src_batch.append(src_ids)
        tgt_batch.append(tgt_ids)

    # Pad sequences
    src_padded = nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in src_batch], padding_value=source_tokenizer.pad_token_id)
    tgt_padded = nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in tgt_batch], padding_value=target_tokenizer.pad_token_id)

    return src_padded.to(device), tgt_padded.to(device)

src_data, tgt_data = prepare_batch(sentence_pairs)


In [ ]:
for epoch in range(num_epochs):
    transformer_model.train()

    optimizer.zero_grad()

    # Prepare input and target
    src = src_data  # Shape: (src_seq_len, batch_size)
    tgt = tgt_data[:-1, :]  # Input to the decoder (excluding the last token)
    tgt_y = tgt_data[1:, :]  # Target output (excluding the first token)

    # Forward pass
    output = transformer_model(src, tgt)
    # Reshape output and target for loss calculation
    output = output.view(-1, output.size(-1))
    tgt_y = tgt_y.contiguous().view(-1)

    # Compute loss
    loss = criterion(output, tgt_y)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/1500], Loss: 10.4254
Epoch [20/1500], Loss: 8.8396
Epoch [40/1500], Loss: 7.9802
Epoch [60/1500], Loss: 7.1947
Epoch [80/1500], Loss: 6.3380
Epoch [100/1500], Loss: 5.3733
Epoch [120/1500], Loss: 4.3884
Epoch [140/1500], Loss: 3.3743
Epoch [160/1500], Loss: 2.5219
Epoch [180/1500], Loss: 1.9103
Epoch [200/1500], Loss: 1.4642
Epoch [220/1500], Loss: 1.1305
Epoch [240/1500], Loss: 0.8858
Epoch [260/1500], Loss: 0.6811
Epoch [280/1500], Loss: 0.5108
Epoch [300/1500], Loss: 0.4256
Epoch [320/1500], Loss: 0.3693
Epoch [340/1500], Loss: 0.2819
Epoch [360/1500], Loss: 0.2538
Epoch [380/1500], Loss: 0.2063
Epoch [400/1500], Loss: 0.1751
Epoch [420/1500], Loss: 0.1507
Epoch [440/1500], Loss: 0.1376
Epoch [460/1500], Loss: 0.1196
Epoch [480/1500], Loss: 0.1012
Epoch [500/1500], Loss: 0.0943
Epoch [520/1500], Loss: 0.0874
Epoch [540/1500], Loss: 0.0891
Epoch [560/1500], Loss: 0.0769
Epoch [580/1500], Loss: 0.0668
Epoch [600/1500], Loss: 0.0633
Epoch [620/1500], Loss: 0.0544
Epoch [640/15

In [ ]:
def translate_sentence(model, sentence, source_tokenizer, target_tokenizer, max_length=20):
    model.eval()
    src_tokens = source_tokenizer.tokenize(sentence)
    src_ids = source_tokenizer.convert_tokens_to_ids(src_tokens)
    src_tensor = torch.tensor(src_ids).unsqueeze(1).to(device)
    tgt_ids = [target_tokenizer.cls_token_id] if target_tokenizer.cls_token_id is not None else [target_tokenizer.pad_token_id]
    tgt_tensor = torch.tensor(tgt_ids).unsqueeze(1).to(device)
    for _ in range(max_length):
        output = model(src_tensor, tgt_tensor)
        last_token_logits = output[-1, 0, :]
        _, next_token = torch.max(last_token_logits, dim=0)
        next_token = next_token.item()
        tgt_tensor = torch.cat([tgt_tensor, torch.tensor([[next_token]]).to(device)], dim=0)
        if next_token == target_tokenizer.sep_token_id or next_token == target_tokenizer.pad_token_id:
            break
    translated_ids = tgt_tensor.squeeze().tolist()
    translated_tokens = target_tokenizer.convert_ids_to_tokens(translated_ids)
    translated_tokens = [token for token in translated_tokens if token not in [target_tokenizer.cls_token, target_tokenizer.sep_token, target_tokenizer.pad_token]]
    translated_sentence = target_tokenizer.convert_tokens_to_string(translated_tokens)
    return translated_sentence

# Example sentences for translation
new_sentences = [
    "Good night.",
    "How old are you?",
    "See you later."
]

for sentence in new_sentences:
    translation = translate_sentence(transformer_model, sentence, source_tokenizer, target_tokenizer)
    print(f"English: {sentence}")
    print(f"French: {translation}")
    print("-" * 50)


English: Good night.
French: ##jour .
--------------------------------------------------
English: How old are you?
French: ##jour , comment ca va ?
--------------------------------------------------
English: See you later.
French: ##jour , comment ca va ?
--------------------------------------------------
